# Summary of Graphical Models and BP

### 1 Graphical models

A graphical model is a compact representation of a collection of probability distributions . It consists of a graph $G = (V, E)$, directed or undirected, where each vertex $v \in V$ is accosiated with a random variable. The edges of the graph represent statistical relationships between the variables. There are several main types of graphical models:

* Bayesian networks
* Markov random fields
* Factor graphs

#### 1.1 Bayesian networks

A Bayesian network is a directed graph where vertices correspond to variables. Each random variable $x_i$ in the network has an associated conditional probability distribution (CPD) or local probabilistic model. A directed edge from $x_i$ to $x_j$ captures a conditional probability of the random variable, given its parents in the graph, namely $P(x_i\ |\ x_j)$. Bayesian network defines a joint probability distribution:

$$
P(x_1, \dots, x_n) = \prod_{i=1}^{n}P(x_i\ |\ \mathbf{Par}(x_i))
$$








#### 1.2 Markov random fields

Markov Random Fields are based on undirected graphical models. As in a Bayesian network, the nodes in the graph represent the variables. An associated probability distribution factorizes into functions, each function associated with a clique of the graph:

$$
P(x_1, ..., x_n) = Z^{-1} \prod_{C \in \mathcal{C}} \psi_C(x_C)
$$

where Z is a constant chosen to ensure that the distribution is normalized. The set $\mathcal{C}$ is often taken to be the set of all maximal cliques of the graph. For a general undirected graph the compatibility functions $\psi_C$ need not have any obvious or direct relation to probabilities or conditional distributions defined over the graph cliques. 

A special case of Markov Random Fields is the pair wise Markov Random Field where the probability distribution factors into functions of two variables and the edges of the graph correspond to constraints between pairs of variables.

#### 1.3 Factor graphs



### 2 Inference in graphical models

### 3 Belief propagation

### 4 Loopy belief propagation

*Useful literature:*

1. D Koller, N Friedman,"Probabilistic Graphical Models: Principles and Techniques", 2009
3. D Koller, N Friedman, L Getoor and B Taskar "Graphical Models in a Nutshell" [[link](https://ai.stanford.edu/~koller/Papers/Koller+al:SRL07.pdf)]
3. J Yedidia, W Freeman, "Understanding belief propagation and its generalizations", 2001 [[link](http://people.csail.mit.edu/billf/publications/Understanding_Belief_Propogation.pdf)]
4. F Pernkopf, R Peharz, S Tschiatschek, "Introduction to Probabilistic Graphical Models", 2014 [[link](https://www.researchgate.net/publication/264040257_Introduction_to_Probabilistic_Graphical_Models)]
5. B Freeman and A Torralba, MIT EECS course 6.869 "Lecture 7: graphical models and belief propagation" [[link](http://helper.ipam.ucla.edu/publications/gss2013/gss2013_11344.pdf)]
6. M Wainwright and M Jordan, "Graphical Models, Exponential Families, and Variational Inference", 2008 [[link](https://people.eecs.berkeley.edu/~wainwrig/Papers/WaiJor08_FTML.pdf)]